In [1]:
import pm4py
from pm4py.objects.log.obj import EventLog, Trace, Event
from demoparser2 import DemoParser
import pandas as pd
from pathlib import Path

In [2]:
parser=DemoParser("heroic-vs-3dmax-m1-dust2.dem")

In [3]:
#parser.parse_event("player_death", player=["last_place_name","team_name"],other=["total_rounds_played","tick"])
parser_attributes = ['total_rounds_played']
b_dfusd_evnts = parser.parse_event("bomb_defused", other=parser_attributes)
b_dfusd_evnts

,site,tick,total_rounds_played,user_name,user_steamid
0,390,48280,5,Lucky,76561198031890115
1,390,96641,10,Ex3rcice,76561198168198200
2,391,129470,13,Alkaren,76561199079856901
3,390,137724,14,nilo,76561198161056567


In [4]:
b_dfusd = b_dfusd_evnts.iloc[0]
r_i = b_dfusd['total_rounds_played']
player = b_dfusd['user_name']
player_id = int(b_dfusd['user_steamid'])
end_t = b_dfusd['tick']

In [5]:
b_plntd_evnts = parser.parse_event("bomb_planted", other=parser_attributes)
b_plntd = b_plntd_evnts[b_plntd_evnts['total_rounds_played'] == r_i].iloc[0]
start_t = b_plntd['tick']

In [6]:
trace = parser.parse_ticks(['last_place_name'], players=[player_id], ticks=range(start_t, end_t + 1))['last_place_name']

In [7]:
change_mask = trace != trace.shift(1)

In [8]:
list(trace[change_mask])

['MidDoors', 'Middle', 'Catwalk', 'ShortStairs', 'ExtendedA', 'BombsiteA']

In [9]:
def entry_to_trace(i: int) -> list[str]:
    b_dfusd = b_dfusd_evnts.iloc[i]
    r_i = b_dfusd['total_rounds_played']
    player = b_dfusd['user_name']
    player_id = int(b_dfusd['user_steamid'])
    end_t = b_dfusd['tick']

    b_plntd = b_plntd_evnts[b_plntd_evnts['total_rounds_played'] == r_i].iloc[0]
    start_t = b_plntd['tick']

    trace = parser.parse_ticks(['last_place_name'], players=[player_id], ticks=range(start_t, end_t + 1))['last_place_name']
    change_mask = trace != trace.shift(1)
    trace = trace[change_mask]
    return list(trace)

In [10]:
traces = []

for i in range(len(b_dfusd_evnts)):
    trace = entry_to_trace(i)
    traces.append(trace)

In [11]:
traces

[['MidDoors', 'Middle', 'Catwalk', 'ShortStairs', 'ExtendedA', 'BombsiteA'],
 ['CTSpawn', 'UnderA', 'LongA', 'ARamp', 'BombsiteA'],
 ['Middle', 'LowerTunnel', 'TunnelStairs', 'UpperTunnel', 'BombsiteB'],
 ['CTSpawn', 'UnderA', 'LongA', 'ARamp', 'BombsiteA']]

In [12]:
target_dir = Path('./Heroric_Dust2_demos') 
absolute_file_paths = [
    str(item.resolve())
    for item in target_dir.iterdir()
    if item.is_file() and 
    str(item.resolve()).endswith('.dem')
]

In [13]:
traces = []
for path in absolute_file_paths:
    print(f"Considering {path}")
    parser=DemoParser(path)
    
    parser_attributes = ['total_rounds_played']
    b_dfusd_evnts = parser.parse_event("bomb_defused", other=parser_attributes)
    b_plntd_evnts = parser.parse_event("bomb_planted", other=parser_attributes)

    count = len(b_dfusd_evnts)

    print(f"  Found {count} bomb defusions!")
    for i in range(count):
        trace = entry_to_trace(i)
        traces.append(trace)
        print(f"  Parsed trace {i+1}")

Considering /home/kochc/projects/process-csgo-star-team/Heroric_Dust2_demos/lp-vs-kru-m1-dust2.dem
  Found 5 bomb defusions!
  Parsed trace 1
  Parsed trace 2
  Parsed trace 3
  Parsed trace 4
  Parsed trace 5
Considering /home/kochc/projects/process-csgo-star-team/Heroric_Dust2_demos/forze-reload-vs-ursa-m2-dust2.dem
  Found 1 bomb defusions!
  Parsed trace 1
Considering /home/kochc/projects/process-csgo-star-team/Heroric_Dust2_demos/heroic-vs-3dmax-m1-dust2.dem
  Found 4 bomb defusions!
  Parsed trace 1
  Parsed trace 2
  Parsed trace 3
  Parsed trace 4
Considering /home/kochc/projects/process-csgo-star-team/Heroric_Dust2_demos/mindfreak-vs-rooster-m1-dust2.dem
  Found 1 bomb defusions!
  Parsed trace 1
Considering /home/kochc/projects/process-csgo-star-team/Heroric_Dust2_demos/liquid-vs-heroic-m2-dust2.dem
  Found 1 bomb defusions!
  Parsed trace 1
Considering /home/kochc/projects/process-csgo-star-team/Heroric_Dust2_demos/heroic-vs-ninjas-in-pyjamas-m2-dust2.dem
  Found 3 bomb defu

In [14]:
event_log = EventLog()
event_log.attributes["concept:name"]= "csgo_demo_log"
for i, log in enumerate(traces):
    trace = Trace()
    trace.attributes["concept:name"] = f"{i}"
    for j, location in enumerate(log):
        event = Event()
        event['concept:name'] = location
        event['concept:activity'] = location
        event['time:tick'] = j
        event['lifecycle:transition'] = "complete"
        trace.append(event)
    event_log.append(trace)
pm4py.write_xes(event_log, "first.xes")

/home/kochc/projects/process-csgo-star-team/.venv/lib/python3.12/site-packages/pm4py/utils.py:992: UserWarning: Install the optional requirement `rustxes` to import/export files faster.
  warnings.warn("Install the optional requirement `rustxes` to import/export files faster.")
/home/kochc/projects/process-csgo-star-team/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 7562.07it/s]
